In [1]:
import jax.numpy as jnp
import jax
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import KeyFunctions as me
import ObjFunc as of
import scipy as sp
import CritPointMR as mr
import torch as pt
from time import time as tmr

In [44]:
#def OPTAccel(MxN, DataSet, EOS):
import torch as pt
from time import time as tmr
import KeyFunctions as me
import ObjFunc as of
import scipy as sp
import CritPointMR as mr
import jax.numpy as jnp
import jax
import numpy as np
import pandas as pd

DataSet = 'DimJiaLi Unique'
EOS = 'PR'
MxNmax = 2

jax.config.update("jax_enable_x64", True)
[y, Tc, Pc, w, C, R, Vc, k, mxNames] = me.LookUpMix(MxN, DataSet, EOS)

default_guess = (jnp.dot(y, Tc).item(), jnp.dot(y, Pc).item())
default_bound = [(default_guess[0], default_guess[0]+100), (default_guess[1], default_guess[1]*2.5)]

t_0 = tmr()
df_minima = sp.optimize.differential_evolution(of.CostFunction, bounds = default_bound, args = (y, Tc, Pc, w, k, R, C, EOS), mutation = 0.95, atol = 1e-14, init = 'sobol')
t_df = tmr()-t_0

path = me.GetRootPath()
path = path+'/3-RootFinding/TrainedModels/MX_'+str(MxN)+'.trh'
trainednn = mr.CompNN(C).to(pt.device("cpu"))
trainednn.load_state_dict(pt.load(path))
trainednn.eval()

ypred = trainednn(pt.tensor(y)).detach().numpy()
PcN = np.dot(np.ones(C)*1/C, Pc)
TcN = np.dot(np.ones(C)*1/C, Tc)
ML_guess = np.multiply(ypred, np.array([TcN, PcN]))
P_plus = ML_guess[1]*1.1
P_minus = ML_guess[1]*0.9

ML_bound = [(ML_guess[0]*.90, ML_guess[0]*1.1), (ML_guess[1]*0.90, ML_guess[1]*1.1)]

t_0 = tmr()
def PlusFunc(T, y, Tc, Pc, w, k, R, C, EOS):
    TP = np.array([T[0], P_plus])
    return of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)

T_plus, f, _, _ = sp.optimize.brute(PlusFunc, ranges = [ML_bound[0]], args = (y, Tc, Pc, w, k, R, C, EOS), full_output=True,)
display(T_plus, f)

def MinusFunc(T, y, Tc, Pc, w, k, R, C, EOS):
    TP = np.array([T[0], P_minus])
    return of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
T_minus, f, _, _ = sp.optimize.brute(PlusFunc, ranges = [ML_bound[0]], args = (y, Tc, Pc, w, k, R, C, EOS), full_output=True,)
display(T_minus, f)

def ConstFunc(T, y, Tc, Pc, w, k, R, C, EOS):
    P = T[0]-T_minus*((P_plus-P_minus)/(T_plus-T_minus))+P_minus
    TP = np.array([T[0], P_minus])
    return of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
bt_x, bt_fval, gr, grout = sp.optimize.brute(ConstFunc, ranges = [ML_bound[0]] ,full_output=True, Ns= 200, args = (y, Tc, Pc, w, k, R, C, EOS), finish=sp.optimize.fmin)

display(gr)
display(grout)

US_conv = df_minima.fun < 1
S_conv = bt_fval < 1

t_frac = (t_df-t_br)/t_df
itr_frac = (df_minima.nfev-400)/df_minima.nfev

    
  #  return US_conv, S_conv, t_frac, itr_frac

array([304.30215925])

633800391.7813118

array([304.30215925])

633800391.7813118

/tmp/ipykernel_194208/1536932393.py:57: RuntimeWarning: divide by zero encountered in divide
  P = T[0]-T_minus*((P_plus-P_minus)/(T_plus-T_minus))+P_minus


array([269.5679789 , 269.86900401, 270.17002911, 270.47105421,
       270.77207931, 271.07310441, 271.37412952, 271.67515462,
       271.97617972, 272.27720482, 272.57822992, 272.87925503,
       273.18028013, 273.48130523, 273.78233033, 274.08335543,
       274.38438054, 274.68540564, 274.98643074, 275.28745584,
       275.58848094, 275.88950605, 276.19053115, 276.49155625,
       276.79258135, 277.09360646, 277.39463156, 277.69565666,
       277.99668176, 278.29770686, 278.59873197, 278.89975707,
       279.20078217, 279.50180727, 279.80283237, 280.10385748,
       280.40488258, 280.70590768, 281.00693278, 281.30795788,
       281.60898299, 281.91000809, 282.21103319, 282.51205829,
       282.81308339, 283.1141085 , 283.4151336 , 283.7161587 ,
       284.0171838 , 284.3182089 , 284.61923401, 284.92025911,
       285.22128421, 285.52230931, 285.82333442, 286.12435952,
       286.42538462, 286.72640972, 287.02743482, 287.32845993,
       287.62948503, 287.93051013, 288.23153523, 288.53

array([1.98463518e+09, 2.00393282e+09, 2.02407743e+09, 2.04512290e+09,
       2.06712716e+09, 2.09015256e+09, 2.11426625e+09, 2.13954063e+09,
       2.16605380e+09, 2.19389011e+09, 2.22314070e+09, 2.25390423e+09,
       2.28628751e+09, 2.32040631e+09, 2.35638630e+09, 2.39436397e+09,
       2.43448775e+09, 2.47691923e+09, 2.52183454e+09, 2.56942586e+09,
       2.61990316e+09, 2.67349612e+09, 2.73045631e+09, 2.79105963e+09,
       2.85560909e+09, 2.92443794e+09, 2.99791320e+09, 3.07643971e+09,
       3.16046472e+09, 3.25048310e+09, 3.34704333e+09, 3.45075437e+09,
       3.56229346e+09, 3.68241523e+09, 3.81196210e+09, 3.95187634e+09,
       4.10321408e+09, 4.26716154e+09, 4.44505403e+09, 4.63839808e+09,
       4.84889748e+09, 5.07848385e+09, 5.32935278e+09, 5.60400658e+09,
       5.90530516e+09, 6.23652673e+09, 6.60144045e+09, 7.00439391e+09,
       7.45041870e+09, 7.94535859e+09, 8.49602569e+09, 9.11039180e+09,
       9.79782406e+09, 1.05693765e+10, 1.14381536e+10, 1.24197645e+10,
      

In [37]:
300.87458952-T_minus*((P_plus-P_minus)/(T_plus-T_minus))+P_minus

/tmp/ipykernel_194208/1220816749.py:1: RuntimeWarning: divide by zero encountered in divide
  300.87458952-T_minus*((P_plus-P_minus)/(T_plus-T_minus))+P_minus


array([-inf])

In [53]:
T_minus

array([304.30215925])

In [ ]:
import pandas as pd
DataSet = 'DimJiaLi Unique'
EOS = 'PR'
MxNmax = 2
df = pd.DataFrame(index = range(MxNmax), columns = ['Unseeded Convergence', 'Seeded Convergence', 'Iteration Reduction', 'Time Reduction'])
for MxN in range(MxNmax):
    US_conv, S_conv, t_frac, itr_frac = OPTAccel(MxN, DataSet, EOS)
    df.iloc[MxN, :] = [US_conv, S_conv, np.round(t_frac*100, 3), np.round(itr_frac*100, 3)]
    df.to_csv('M2CompAccel.csv')
    

from time import time as t
t0= t()
[y, Tc, Pc, w, C, R, Vc, k, mxNames] = me.LookUpMix(MxN, DataSet, EOS)
TP_guess = [4.41044394e+02, 6.25409061e+06]
def ModCostFunc(T, P, y, Tc, Pc, w, k, R, C, EOS):
    TP = np.array([T[0], P])
    F = of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
    return F

P_max = TP_guess[1]
P_min = TP_guess[1]*0.90
T_max = sp.optimize.brute(ModCostFunc, [(418.9921740807772, 463.0966134577012)], args = (P_max, y, Tc, Pc, w, k, R, C, EOS), Ns= 5)
T_min =  sp.optimize.brute(ModCostFunc, [(418.9921740807772, 463.0966134577012)], args = (P_min, y, Tc, Pc, w, k, R, C, EOS), Ns= 5)

def ModCostFunc(TP, y, Tc, Pc, w, k, R, C, EOS):
    TP[0] = (P_max-P_min)/(T_max-T_min)*(TP[0]-T_max)+P_max
    F = of.CostFunction(TP, y, Tc, Pc, w, k, R, C, EOS)
    return F

bt_x, bt_fval,_,_ = sp.optimize.brute(ModCostFunc, ranges = [(418.9921740807772, 463.0966134577012),
 (5941386.081873446, 6566795.143123283)], full_output=True, args = (y, Tc, Pc, w, k, R, C, EOS), finish=sp.optimize.fmin, Ns= 5)
t_1 =t()-t0
display(t_1)
display(bt_fval)

In [4]:
t_frac

0.902835714005123